### Pytorch



In [ ]:
# Importación de librerías esenciales
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Verificación de la disponibilidad de GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Dispositivo usado:", device)


## 1. Grafo computacional dinámico
# Ejemplo simple de grafo computacional dinámico
x = torch.tensor([2.0], requires_grad=True)
y = x * 3       # Operación multiplicativa
z = y ** 2      # Operación de potencia
z.backward()    # Propagación hacia atrás: cálculo de gradientes

print("Valor de x:", x.item())
print("Gradiente de x:", x.grad.item())
# Se espera que el gradiente sea calculado como d(z)/d(x) = 2*y*dy/dx = 2*(3x)*3 = 18 cuando x=2

## 2. Visualización de grafos computacionales

# Descomentar la siguiente línea si torchviz no está instalado:
!pip install torchviz

from torchviz import make_dot

# Ejemplo para visualizar el grafo computacional
x = torch.tensor([2.0], requires_grad=True)
y = x * 3
z = y ** 2
dot = make_dot(z, params={'x': x})
# Se guarda la visualización en un archivo 'grafo_computacional.png'
dot.render("grafo_computacional", format="png")
print("Grafo computacional guardado en 'grafo_computacional.png'")


## 3. Tensores y propiedades avanzadas

# Creación de un tensor simple en CPU
tensor_cpu = torch.tensor([[1, 2], [3, 4]], dtype=torch.float32)
print("Tensor en CPU:")
print(tensor_cpu)

# Mover el tensor a GPU si está disponible
if torch.cuda.is_available():
    tensor_gpu = tensor_cpu.to('cuda')
    print("Tensor en GPU:")
    print(tensor_gpu)
else:
    print("GPU no disponible. Se continuará trabajando en CPU.")


## 4. Técnicas de slicing y reshape
# Creación de un tensor de ejemplo con 20 elementos
tensor = torch.arange(1, 21).reshape(4, 5)
print("Tensor original (4x5):")
print(tensor)

# Slicing: extraer las dos primeras filas y tres columnas
subtensor = tensor[:2, :3]
print("Subtensor obtenido mediante slicing (2x3):")
print(subtensor)

# Cambio de forma usando reshape: transformar a una matriz de 5x4
tensor_reshaped = tensor.reshape(5, 4)
print("Tensor redimensionado a (5x4):")
print(tensor_reshaped)

## 5. Autograd y extensibilidad

# Ejemplo básico con autograd
a = torch.tensor([3.0], requires_grad=True)
b = torch.tensor([4.0], requires_grad=True)
c = a * b + a ** 2
c.backward()

print("Gradiente de a (calculado automáticamente):", a.grad.item())
print("Gradiente de b (calculado automáticamente):", b.grad.item())

# Ejemplo de función autograd personalizada
class MiFuncionAutograd(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        # Guardamos el tensor de entrada para usarlo en backward
        ctx.save_for_backward(input)
        # Operación forward: elevar al cuadrado el input
        return input ** 2

    @staticmethod
    def backward(ctx, grad_output):
        # Recuperamos el tensor de entrada almacenado
        input, = ctx.saved_tensors
        # Cálculo del gradiente: derivada de input**2 es 2*input
        grad_input = grad_output * 2 * input
        return grad_input

# Uso de la función personalizada
x = torch.tensor([5.0], requires_grad=True)
y = MiFuncionAutograd.apply(x)
y.backward()
print("Gradiente de x con función personalizada:", x.grad.item())

"""
## 6. Algoritmos de backpropagation

El algoritmo de backpropagation es fundamental en el entrenamiento de redes neuronales, ya que permite ajustar los parámetros del modelo mediante el cálculo de gradientes. El proceso involucra:
1. **Forward pass:** Calcular la salida del modelo a partir de la entrada.
2. **Cálculo de la pérdida:** Evaluar qué tan lejos está la predicción del valor esperado.
3. **Backward pass:** Calcular los gradientes de la pérdida respecto a cada parámetro utilizando la regla de la cadena.
4. **Actualización de parámetros:** Utilizar un optimizador para actualizar los pesos en función de los gradientes calculados.
"""

# Definición de una red neuronal simple
class RedSimple(nn.Module):
    def __init__(self):
        super(RedSimple, self).__init__()
        self.fc = nn.Linear(2, 1)  # Capa lineal: 2 entradas -> 1 salida

    def forward(self, x):
        return self.fc(x)

# Instanciación del modelo, definición de la función de pérdida y del optimizador
modelo = RedSimple()
criterion = nn.MSELoss()
optimizer = optim.SGD(modelo.parameters(), lr=0.01)

# Datos de entrenamiento (entrada y salida deseada)
entradas = torch.tensor([[1.0, 2.0],
                         [2.0, 3.0],
                         [3.0, 4.0]])
salidas = torch.tensor([[3.0],
                        [5.0],
                        [7.0]])

# Ciclo de entrenamiento
for epoch in range(100):
    optimizer.zero_grad()          # Reiniciar gradientes
    predicciones = modelo(entradas)
    loss = criterion(predicciones, salidas)
    loss.backward()                # Backward pass: cálculo de gradientes
    optimizer.step()               # Actualización de parámetros

print("Pérdida final tras entrenamiento (RedSimple con SGD):", loss.item())


## 7. Máscaras (Masks)

# Ejemplo de enmascaramiento en secuencias
secuencias = torch.tensor([[1, 2, 0, 0],
                           [3, 4, 5, 0]])
mascara = (secuencias != 0)  # True donde el valor no es 0
print("Secuencias originales:")
print(secuencias)
print("Máscara generada (True indica valores no nulos):")
print(mascara)

# Uso de la máscara para extraer únicamente los elementos válidos
elementos_filtrados = secuencias[mascara]
print("Elementos filtrados utilizando la máscara:")
print(elementos_filtrados)

## 8. Position-wise feed-forward networks

# Definición de la position-wise feed-forward network
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Primera transformación seguida de una activación ReLU
        out = F.relu(self.fc1(x))
        # Aplicación de dropout y segunda transformación lineal
        out = self.fc2(self.dropout(out))
        return out

# Parámetros para el ejemplo
d_model = 512
d_ff = 2048
ffn = PositionwiseFeedForward(d_model, d_ff)

# Ejemplo: se asume que la entrada 'x' tiene forma [batch_size, seq_len, d_model]
x = torch.randn(10, 20, d_model)
output = ffn(x)
print("Forma de la salida de la position-wise feed-forward network:", output.shape)


## 9. Manejo de batches (división de conjunto de datos)

from torch.utils.data import DataLoader, TensorDataset

# Creación de un dataset sintético
datos = torch.randn(100, 10)           # 100 muestras, 10 características cada una
etiquetas = torch.randint(0, 2, (100,))  # 100 etiquetas binarias

dataset = TensorDataset(datos, etiquetas)
loader = DataLoader(dataset, batch_size=16, shuffle=True)

# Iteración sobre los batches
for batch_idx, (batch_datos, batch_etiquetas) in enumerate(loader):
    print(f"Batch {batch_idx}:")
    print("  Datos:", batch_datos.shape)
    print("  Etiquetas:", batch_etiquetas.shape)
    # Aquí se podría incluir el proceso de forward y backward de un modelo

## 10. Optimizadores

# Reutilizamos la clase RedSimple definida anteriormente
modelo = RedSimple()
criterion = nn.MSELoss()
optimizer = optim.Adam(modelo.parameters(), lr=0.001)

# Datos de ejemplo
entradas = torch.tensor([[1.0, 2.0],
                         [2.0, 3.0],
                         [3.0, 4.0]])
salidas = torch.tensor([[3.0],
                        [5.0],
                        [7.0]])

# Ciclo de entrenamiento simple
for epoch in range(100):
    optimizer.zero_grad()
    predicciones = modelo(entradas)
    loss = criterion(predicciones, salidas)
    loss.backward()
    optimizer.step()

print("Pérdida final usando Adam:", loss.item())


## 11. Label Smoothing
def label_smoothing_loss(pred, target, smoothing=0.1):
    """
    Calcula la pérdida con label smoothing.
    
    Parámetros:
      - pred: tensor con logits de predicción (sin aplicar softmax)
      - target: tensor con índices de la clase correcta
      - smoothing: factor de suavizado (por defecto 0.1)
    
    La función crea una distribución "suavizada" en la que se asigna la mayor probabilidad a la clase correcta y se distribuye el resto entre las demás clases.
    """
    num_classes = pred.size(1)
    # Crear una distribución de etiquetas suavizadas
    with torch.no_grad():
        true_dist = torch.zeros_like(pred)
        true_dist.fill_(smoothing / (num_classes - 1))
        # Asignar la probabilidad correspondiente a la etiqueta correcta
        true_dist.scatter_(1, target.data.unsqueeze(1), 1.0 - smoothing)
    # Cálculo de la pérdida (entropía cruzada suavizada)
    return torch.mean(torch.sum(-true_dist * F.log_softmax(pred, dim=1), dim=1))

# Ejemplo de uso del label smoothing en un batch de predicciones
predicciones = torch.randn(3, 5)  # 3 muestras, 5 clases
etiquetas = torch.tensor([1, 0, 4])
loss_ls = label_smoothing_loss(predicciones, etiquetas, smoothing=0.1)
print("Pérdida con label smoothing:", loss_ls.item())

## 12. Batch normalization
# Ejemplo de batch normalization para tensores de 1 dimensión (características)
batch_norm = nn.BatchNorm1d(10)  # Normalización para 10 características
x = torch.randn(16, 10)         # 16 muestras, 10 características cada una
x_normalized = batch_norm(x)
print("Salida tras aplicar Batch Normalization:")
print(x_normalized)

## 13. Dropout y dropout variacional
# Ejemplo básico de Dropout en una capa lineal
dropout = nn.Dropout(p=0.5)
x = torch.randn(5, 10)
x_dropout = dropout(x)
print("Salida después de aplicar Dropout (p=0.5):")
print(x_dropout)

# Ejemplo sencillo de dropout variacional aplicado a una red recurrente
class RNNConDropoutVariacional(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout=0.5):
        super(RNNConDropoutVariacional, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        # x: [batch_size, seq_len, input_size]
        output, hidden = self.rnn(x)
        # Aplicar dropout variacional: la misma máscara se aplica a todas las salidas
        output = self.dropout(output)
        return output, hidden

# Ejemplo de uso de la RNN con dropout variacional
rnn_model = RNNConDropoutVariacional(input_size=10, hidden_size=20, num_layers=2, dropout=0.5)
x_seq = torch.randn(8, 15, 10)  # 8 muestras, secuencia de longitud 15, 10 características
output, hidden = rnn_model(x_seq)
print("Forma de la salida de la RNN con dropout variacional:", output.shape)

## 14. Skip connections y residual connections
# Implementación de un bloque residual simple
class BloqueResidual(nn.Module):
    def __init__(self, in_features):
        super(BloqueResidual, self).__init__()
        self.fc1 = nn.Linear(in_features, in_features)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(in_features, in_features)
    
    def forward(self, x):
        identidad = x         # Guardamos la entrada original para la conexión residual
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        # Suma de la entrada original y la transformación
        out += identidad
        out = self.relu(out)
        return out

# Ejemplo de uso del bloque residual
x = torch.randn(5, 10)  # 5 muestras, 10 características cada una
bloque = BloqueResidual(10)
salida_residual = bloque(x)
print("Forma de la salida del bloque residual:", salida_residual.shape)


### Ejercicios

#### Ejercicio 1: Propiedades básicas y avanzadas de los tensores

**Objetivo:**  
- Familiarizarse con la creación, manipulación y operación sobre tensores en PyTorch, comprendiendo tanto propiedades básicas (dimensiones, tipos, dispositivo) como operaciones avanzadas (broadcasting, indexación avanzada, manipulación de formas).

**Instrucciones:**  
- Definir tensores de distintas dimensiones y tipos de datos.
- Realizar operaciones aritméticas (suma, multiplicación, exponenciación) y comprobar las reglas de broadcasting.
- Explorar propiedades como la mutabilidad, la conversión entre dispositivos (CPU/GPU) y el uso de métodos que permiten modificar la forma sin copiar datos (por ejemplo, métodos para transponer o permutar ejes).

**Preguntas:**  
- ¿Qué diferencias existen entre un tensor en CPU y uno en GPU en términos de eficiencia computacional?
- ¿Cómo afecta el broadcasting en operaciones element-wise y qué precauciones se deben tomar?
- ¿Qué ventajas ofrece la manipulación de formas (reshape) respecto a la copia de datos en memoria?


#### Ejercicio 2: Construcción y visualización del grafo computacional dinámico

**Objetivo:**  
- Comprender el concepto de grafo computacional dinámico y la importancia del seguimiento de operaciones para la diferenciación automática.

**Instrucciones:**  
- Diseñar una secuencia de operaciones con tensores que involucren operaciones aritméticas y funciones de activación.
- Habilitar la construcción del grafo computacional mediante la asignación de `requires_grad=True` en los tensores.
- Investigar y describir un método para visualizar este grafo (por ejemplo, mediante una herramienta de visualización) y explicar la relevancia de cada nodo y sus conexiones en el contexto del cálculo del gradiente.

**Preguntas:**  
- ¿Cómo se construye el grafo en tiempo real y qué significa que sea “dinámico”?
- ¿Qué información ofrece la visualización y cómo se relaciona con la cadena de derivadas en backpropagation?
- ¿Qué ventajas ofrece este enfoque en comparación con los métodos de diferenciación simbólica?


#### Ejercicio 3: Técnicas de slicing, reshape y manipulación de datos en tensores

**Objetivo:**  
- Practicar la extracción y reorganización de datos a partir de tensores, fundamental para la preparación y el preprocesamiento de datos en IA.

**Instrucciones:**  
- Partir de un tensor de dimensiones conocidas y aplicar técnicas de slicing para extraer sub-tensores específicos.
- Cambiar la forma de un tensor utilizando técnicas de reshape, observando cómo se reordenan los elementos.
- Analizar la diferencia entre operaciones que generan vistas versus aquellas que crean nuevos tensores, y discutir la importancia de esta distinción en términos de memoria y rendimiento.

**Preguntas:**  
- ¿En qué situaciones es preferible utilizar slicing en lugar de copiar datos?
- ¿Qué consideraciones se deben tener al redimensionar datos para que sean compatibles con un modelo?
- ¿Cómo se gestionan los errores comunes al cambiar la forma de los tensores?

#### Ejercicio 4: Autograd - diferenciación automática y funciones personalizadas

**Objetivo:**  
- Profundizar en el mecanismo de diferenciación automática de PyTorch y aprender a implementar funciones personalizadas que definan sus propias reglas de forward y backward.

**Instrucciones:**  
- Describir el proceso de cálculo del gradiente mediante la regla de la cadena en un escenario simple.
- Diseñar una función teórica que realice una operación matemática no estándar y que requiera definir manualmente la función de backward.
- Comparar el resultado de la diferenciación automática con el cálculo manual del gradiente, identificando posibles discrepancias o casos especiales.

**Preguntas:**  
- ¿Por qué es necesario almacenar información en el contexto (ctx) durante el forward?
- ¿Cuáles son los desafíos y beneficios de crear una función personalizada en comparación con las operaciones predefinidas?
- ¿Cómo se puede validar que la implementación del backward es correcta?

#### Ejercicio 5: Extensibilidad de PyTorch mediante módulos y funciones personalizadas

**Objetivo:**  
- Explorar la capacidad de PyTorch para definir módulos y funciones que integren algoritmos de backpropagation, permitiendo la creación de arquitecturas de red personalizadas.

**Instrucciones:**  
- Elaborar un diseño teórico para un módulo personalizado que incluya capas lineales, activaciones y una función de pérdida específica.
- Discutir cómo se integran estas funciones personalizadas en el ciclo de entrenamiento (forward, cálculo de pérdida, backward y actualización de parámetros).
- Proponer mejoras o modificaciones al módulo que puedan facilitar la interpretación del gradiente o la eficiencia del entrenamiento.

**Preguntas:**  
- ¿Qué ventajas ofrece la creación de módulos personalizados en términos de modularidad y reutilización del código?
- ¿Cómo se garantiza que la función de pérdida personalizada se integre adecuadamente en el proceso de backpropagation?
- ¿Qué problemas podrían surgir al extender las capacidades de PyTorch y cómo se podrían solucionar?


#### Ejercicio 6: Uso y aplicación de máscaras en datos secuenciales

**Objetivo:**  
- Comprender el uso de máscaras para filtrar información irrelevante o nula en datos secuenciales, fundamental en tareas de procesamiento de lenguaje natural y secuencias.

**Instrucciones:**  
- Analizar un escenario en el que se disponga de secuencias con elementos de relleno (padding) y definir una estrategia para identificar y excluir dichos elementos.
- Discutir la implementación teórica de una máscara que permita extraer únicamente los elementos válidos de la secuencia.
- Evaluar el impacto de la aplicación de la máscara en el entrenamiento y en la predicción de modelos secuenciales.

**Preguntas:**  
- ¿Por qué es crucial eliminar o ignorar elementos de relleno en modelos secuenciales?
- ¿Cómo se puede garantizar que la máscara se aplique de manera consistente durante el entrenamiento y la inferencia?
- ¿Qué otros escenarios de datos podrían beneficiarse del uso de máscaras?


#### Ejercicio 7: Implementación de position-wise feed-forward networks

**Objetivo:**  
- Investigar el funcionamiento y la aplicación de las Position-wise Feed-Forward Networks, esenciales en arquitecturas Transformer.

**Instrucciones:**  
- Describir el flujo de datos en una red feed-forward que se aplica de forma independiente a cada posición de una secuencia.
- Analizar la función de cada componente: capas lineales, activación (por ejemplo, ReLU) y dropout.
- Evaluar, de forma teórica, cómo esta arquitectura permite el procesamiento paralelo de secuencias y mejora la eficiencia del modelo.

**Preguntas:**  
- ¿Qué beneficios ofrece el procesamiento posición por posición en comparación con otras arquitecturas?
- ¿Cómo se integra esta red dentro de un modelo Transformer y qué impacto tiene en el rendimiento?
- ¿Qué limitaciones podrían existir al utilizar este enfoque en contextos de datos muy heterogéneos?


#### Ejercicio 8: Manejo de batches y uso de optimizadores en el entrenamiento

**Objetivo:**  
- Comprender la importancia del manejo de batches en el entrenamiento de modelos y la influencia de diferentes optimizadores en la convergencia del modelo.

**Instrucciones:**  
- Diseñar un procedimiento teórico para dividir un conjunto de datos en batches y discutir cómo afecta el tamaño de batch al cálculo del gradiente.
- Comparar las características y comportamiento de al menos dos optimizadores (por ejemplo, Adam y SGD) en un escenario de entrenamiento.
- Reflexionar sobre la elección del optimizador adecuado para distintos tipos de modelos y conjuntos de datos.

**Preguntas:**  
- ¿Qué ventajas y desventajas tiene el uso de batches pequeños versus batches grandes?
- ¿Cómo afecta la tasa de aprendizaje la convergencia de cada optimizador?
- ¿Qué estrategias se pueden implementar para mejorar la estabilidad del entrenamiento en función del optimizador elegido?


#### Ejercicio 9: Aplicación de técnicas de label smoothing

**Objetivo:**  
- Explorar la técnica de label smoothing para suavizar las etiquetas en problemas de clasificación, reduciendo la sobreconfianza del modelo y mejorando la generalización.

**Instrucciones:**  
- Definir, en términos teóricos, qué es label smoothing y cómo se modifica la distribución de probabilidad de las etiquetas.
- Comparar los efectos de entrenar un modelo con y sin label smoothing en un escenario de clasificación.
- Analizar posibles casos en los que el label smoothing puede ayudar a evitar el sobreajuste y mejorar la robustez del modelo.

**Preguntas:**  
- ¿Qué ventajas ofrece suavizar las etiquetas en comparación con utilizar etiquetas “hard”?
- ¿Cómo se debe ajustar el parámetro de suavizado para evitar la pérdida de información crítica?
- ¿Qué impacto tiene esta técnica en la interpretación de las salidas del modelo?


#### Ejercicio 10: Integración de batch normalization y dropout en redes profundas

**Objetivo:**  
- Investigar cómo la normalización de batches y el dropout pueden mejorar la estabilidad y el rendimiento del entrenamiento en redes profundas.

**Instrucciones:**  
- Describir la función de Batch Normalization y cómo normaliza las activaciones de cada capa durante el entrenamiento.
- Analizar el papel del Dropout en la reducción del sobreajuste, explicando el mecanismo de “apagado” aleatorio de neuronas.
- Diseñar, a nivel conceptual, una arquitectura profunda que combine ambas técnicas y discutir cómo interactúan para mejorar la convergencia.

**Preguntas:**  
- ¿Cómo afecta la normalización en cada capa a la propagación del gradiente en redes muy profundas?
- ¿Qué aspectos se deben considerar al combinar Batch Normalization y Dropout en una misma arquitectura?
- ¿En qué situaciones podría ser contraproducente aplicar ambas técnicas simultáneamente?


#### Ejercicio 11: Uso de dropout variacional en arquitecturas recurrentes

**Objetivo:**  
- Comprender la implementación y los beneficios del dropout variacional en redes recurrentes, asegurando que la máscara de dropout sea coherente a lo largo de la secuencia.

**Instrucciones:**  
- Explicar la diferencia entre el dropout estándar y el dropout variacional en el contexto de redes recurrentes.
- Describir, teóricamente, cómo se aplica una máscara de dropout que se mantenga constante en cada paso temporal.
- Evaluar las ventajas que aporta esta técnica en términos de retención de información y estabilidad en la propagación a través del tiempo.

**Preguntas:**  
- ¿Por qué es importante mantener una máscara consistente en arquitecturas recurrentes?
- ¿Qué impacto tiene el dropout variacional en la capacidad del modelo para capturar dependencias a largo plazo?
- ¿Cuáles son los desafíos al ajustar la tasa de dropout en este contexto?


#### Ejercicio 12: Implementación de skip connection y conexiones residuales

**Objetivo:**  
- Analizar y aplicar el concepto de conexiones residuales para facilitar el entrenamiento de modelos profundos y mitigar el problema del desvanecimiento del gradiente.

**Instrucciones:**  
- Describir el concepto de skip connections y cómo permiten la propagación directa de la información.
- Plantear, de manera teórica, la implementación de un bloque residual que incluya transformaciones intermedias y la suma de la entrada original.
- Discutir las ventajas y posibles inconvenientes de utilizar conexiones residuales en diferentes tipos de arquitecturas (por ejemplo, redes convolucionales vs. redes recurrentes).

**Preguntas:**  
- ¿Cómo ayudan las conexiones residuales a mejorar la propagación del gradiente en redes muy profundas?
- ¿Qué diferencias existen entre implementar skip connections de forma simple y usar bloques residuales más complejos?
- ¿En qué escenarios podría ser especialmente beneficioso incluir estas conexiones en un modelo?


#### Ejercicio 13: Exploración de otras técnicas avanzadas en PyTorch para IA

**Objetivo:**  
- Investigar y analizar otras técnicas avanzadas que ofrece PyTorch y que resultan útiles en aplicaciones de inteligencia artificial.

**Instrucciones:**  
- Investigar y documentar al menos dos técnicas adicionales, por ejemplo:
  - **Atención multicabecera:** Analizar cómo se implementa la atención en modelos de procesamiento de lenguaje y visión, y discutir su impacto en la eficiencia del modelo.
  - **Embeddings y representación de datos:** Estudiar el uso de capas de embeddings para representar datos categóricos o secuenciales de forma densa.
  - **Learning rate schedulers:** Explorar estrategias para ajustar dinámicamente la tasa de aprendizaje durante el entrenamiento y su efecto en la convergencia.
  - **Data augmentation para imágenes o secuencias:** Discutir técnicas para aumentar el tamaño y la diversidad de los datos de entrenamiento.
- Elaborar un informe teórico que incluya la definición, el funcionamiento y el impacto de cada técnica en modelos de IA.

**Preguntas:**  
- ¿Cómo mejora cada técnica la capacidad de un modelo para generalizar a nuevos datos?
- ¿Qué consideraciones se deben tener al implementar estas técnicas en entornos de producción?
- ¿Cómo se pueden combinar estas técnicas con las ya estudiadas para construir modelos más robustos y eficientes?


In [ ]:
## Tus respuestas